In [97]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
from datetime import datetime, date, timedelta

# statistical analysis
import statsmodels.api as sm
from statsmodels import regression

import yfinance as yfin
yfin.pdr_override()

import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

#from var import VaR

pd.set_option('display.float_format', lambda x: f'{x:,.4f}')
sns.set_theme(rc={'axes.formatter.limits': (-8, 9)})
style.use('seaborn-whitegrid')
plt.rcParams["figure.figsize"] = (10,6)

In [98]:
start_date = datetime(2022, 1, 1)
purchase_date = datetime(2023, 9, 11)
purchase_date_str = '2023-9-12'
end_date = datetime.now()
end_date_str = end_date.strftime("%Y-%m-%d")

alpha = 0.05
period = '5y'

folio_df = pd.read_csv('https://raw.githubusercontent.com/dsimband/DATA618/main/w7/data/DATA618_Portfolio.csv', 
                 dtype={
                    'ID': 'int',
                    'Price': 'float',
                    'Shares': 'float',
                    'Value': 'float', 
                 })

In [99]:
folio_df = pd.read_csv('./data/DATA618_Portfolio.csv', 
                 dtype={
                    'ID': 'int',
                    'Price': 'float',
                    'Shares': 'float',
                    'Value': 'float', 
                 })

folio_df = folio_df[folio_df['Shares'] > 0]
folio_df = folio_df.groupby(['Ticker','BondName','Class','Sub_Class'])[['Shares','Value']].sum()
folio_df.reset_index(inplace=True)

# calculate portfilio percentage
portfolio_total = folio_df['Value'].sum()
folio_df['port_percent'] = folio_df['Value'] / portfolio_total



In [100]:
folio_df

,Ticker,BondName,Class,Sub_Class,Shares,Value,port_percent
0,ALTVX,AB Municipal Income National Advisor,Interest Rate Sensitive,Very-High-Quality Municipal Bonds,"106,383.0000","1,000.0000",0.0120
1,BLUEX,AMG Veritas Global Real Return I,Economically Sensitive,Long/Short Equities,"14,201.0000",500.0000,0.0060
2,BPLSX,Boston Partners Long/Short Equity Instl,Economically Sensitive,Long/Short Equities,"49,967.0000",750.0000,0.0090
3,C_A_S_H,Cash,Cash,Cash,"4,182,856.0000","4,183.0000",0.0501
4,DFAR,Dimensional US Real Estate ETF,Economically Sensitive,Publicly Traded REITs,"108,467.0000","2,250.0000",0.0270
5,FREL,Fidelity MSCI Real Estate ETF,Economically Sensitive,Publicly Traded REITs,"115,479.0000","2,800.0000",0.0336
6,FSMD,Fidelity Small-Mid Multifactor ETF,Economically Sensitive,U.S. Equities,"120,705.0000","4,000.0000",0.0479
7,GARIX,Gotham Absolute Return Institutional,Economically Sensitive,Long/Short Equities,"25,278.0000",500.0000,0.0060
8,GCHDX,Gotham Hedged Core Institutional,Economically Sensitive,Long/Short Equities,"46,211.0000",500.0000,0.0060
9,ICBFX,Invesco Corporate Bond R6,Interest Rate Sensitive,Very-High-Quality Corporate Bonds,"166,389.0000","1,000.0000",0.0120


In [101]:
# ticker symbols
ticker_lst = list(folio_df['Ticker'])
print('ticker #:', len(ticker_lst))


# portfolio weights
weight_lst = (folio_df['port_percent'].values)
print('price #:', len(weight_lst))


#Download closing prices
price_df = pdr.get_data_yahoo(ticker_lst, start=purchase_date)['Close']

price_df['C_A_S_H'] = 1
print('price_df #:', price_df.shape)

ticker #: 35
price #: 35
[*********************100%%**********************]  35 of 35 completed


1 Failed download:
['C_A_S_H']: Exception('%ticker%: No timezone found, symbol may be delisted')



price_df #: (24, 35)


In [102]:
t_df = price_df[:1].T
t_df.reset_index(inplace=True)
t_df.columns = ['Ticker','Price']
t_df

,Ticker,Price
0,ALTVX,9.4000
1,BLUEX,35.2100
2,BPLSX,15.0100
3,C_A_S_H,1.0000
4,DFAR,20.9600
5,FREL,24.4700
6,FSMD,33.2600
7,GARIX,19.7800
8,GCHDX,10.8200
9,ICBFX,6.0100


In [103]:
merge_df = folio_df.merge(t_df, how='outer', left_on='Ticker', right_on='Ticker')
merge_df['test'] = round(merge_df['Value'] *1000 / merge_df['Price'],0)
merge_df['test2'] = merge_df['Shares'] * merge_df['Price']
merge_df

,Ticker,BondName,Class,Sub_Class,Shares,Value,port_percent,Price,test,test2
0,ALTVX,AB Municipal Income National Advisor,Interest Rate Sensitive,Very-High-Quality Municipal Bonds,"106,383.0000","1,000.0000",0.0120,9.4000,"106,383.0000","1,000,000.1594"
1,BLUEX,AMG Veritas Global Real Return I,Economically Sensitive,Long/Short Equities,"14,201.0000",500.0000,0.0060,35.2100,"14,201.0000","500,017.1970"
2,BPLSX,Boston Partners Long/Short Equity Instl,Economically Sensitive,Long/Short Equities,"49,967.0000",750.0000,0.0090,15.0100,"49,967.0000","750,004.6814"
3,C_A_S_H,Cash,Cash,Cash,"4,182,856.0000","4,183.0000",0.0501,1.0000,"4,183,000.0000","4,182,856.0000"
4,DFAR,Dimensional US Real Estate ETF,Economically Sensitive,Publicly Traded REITs,"108,467.0000","2,250.0000",0.0270,20.9600,"107,347.0000","2,273,468.2207"
5,FREL,Fidelity MSCI Real Estate ETF,Economically Sensitive,Publicly Traded REITs,"115,479.0000","2,800.0000",0.0336,24.4700,"114,426.0000","2,825,771.0507"
6,FSMD,Fidelity Small-Mid Multifactor ETF,Economically Sensitive,U.S. Equities,"120,705.0000","4,000.0000",0.0479,33.2600,"120,265.0000","4,014,648.0974"
7,GARIX,Gotham Absolute Return Institutional,Economically Sensitive,Long/Short Equities,"25,278.0000",500.0000,0.0060,19.7800,"25,278.0000","499,998.8574"
8,GCHDX,Gotham Hedged Core Institutional,Economically Sensitive,Long/Short Equities,"46,211.0000",500.0000,0.0060,10.8200,"46,211.0000","500,003.0059"
9,ICBFX,Invesco Corporate Bond R6,Interest Rate Sensitive,Very-High-Quality Corporate Bonds,"166,389.0000","1,000.0000",0.0120,6.0100,"166,389.0000","999,997.9281"


In [104]:
merge_df['test2'].sum()

83718683.57596111

In [105]:
merge_df['Value'].sum()

83433.0

In [106]:
merge_df['test2'].sum() - merge_df['Value'].sum()

83635250.57596111

In [107]:
merge_df['Value'].sum() * 1000

83433000.0

In [108]:
merge_df['test2'].sum() - merge_df['Value'].sum() * 1000

285683.575961113

In [109]:
4469000.0000 - 285827.575961113

4183172.424038887

In [110]:
44183172 / 1000

44183.172

In [111]:
83954144 - 83719000

235144

In [112]:
4418000 - 235144

4182856